In [6]:
from nltk.tokenize import word_tokenize
import os
import pandas as pd


df = pd.read_csv(os.getcwd() + '/../data/all_text.csv', encoding = 'iso-8859-1')

# top chacters
top_n_characters = 20

In [7]:
# drops lines that aren't dialog
df['drop'] = 0
df.ix[df['text'].str[0] == '[', 'drop'] = 1
df = df.ix[df['drop'] == 0, :]
df.head()

,episode,text,drop
1,1,"Pen: Jake! Jake! Good mornin'! Whatcha doin',...",0
2,1,Jake: Meditating.\r\n,0
3,1,"Pen: What are you meditating about, dude?\r\n",0
4,1,"Jake: Wait, dude. I'm connecting to the inter...",0
6,1,Pen: Ha ha! Beep boop boop boop beep boop.\r\n,0


In [8]:
# modifies the data set
df['character'] = ''

txt = 'Pen: just a test'
for i, row in df.iterrows():
    txt = row['text']
    df.ix[i, 'character'] = txt[:txt.find(':')].strip()
    df.ix[i, 'text'] = txt[txt.find(':') + 1:].strip()

df.head()

,episode,text,drop,character
1,1,"Jake! Jake! Good mornin'! Whatcha doin', buddy?",0,Pen
2,1,Meditating.,0,Jake
3,1,"What are you meditating about, dude?",0,Pen
4,1,"Wait, dude. I'm connecting to the internet wit...",0,Jake
6,1,Ha ha! Beep boop boop boop beep boop.,0,Pen


In [9]:
# corrects the episodes
df['episode'] = df['episode'] - 1

df.head()

,episode,text,drop,character
1,0,"Jake! Jake! Good mornin'! Whatcha doin', buddy?",0,Pen
2,0,Meditating.,0,Jake
3,0,"What are you meditating about, dude?",0,Pen
4,0,"Wait, dude. I'm connecting to the internet wit...",0,Jake
6,0,Ha ha! Beep boop boop boop beep boop.,0,Pen


In [10]:
# gets the number of lines each character has
df['count'] = 1
df_vocab = df.ix[:, ['character', 'count']].groupby('character').sum()
df_vocab = df_vocab.reset_index()
df_vocab = df_vocab.sort_values('count', ascending = False)

# gets the vocab size of each character
character_list = df_vocab.ix[:top_n_characters, 'character']
df_vocab = df_vocab.ix[:top_n_characters, :].reset_index(drop = True)

# renames some columns
df_vocab['vocab'] = 0
df_vocab.columns = ['character', 'lines', 'vocab']

for character in character_list:
    vocab = df.ix[df['character'] == character, 'text'].tolist()
    vocab = ' '.join(vocab) 
    vocab = word_tokenize(vocab)
    vocab = set(vocab)
    df_vocab.ix[df_vocab['character'] == character, 'vocab'] = len(vocab)
    
df_vocab.head()

,character,lines,vocab
0,Finn,6576,8257
1,Jake,4757,7210
2,Princess Bubblegum,1135,3364
3,Ice King,919,3815
4,Marceline,481,1647


In [11]:
df_vocab['vocab_ratio'] = df_vocab['vocab'] / df_vocab['lines']
df_vocab.head()

,character,lines,vocab,vocab_ratio
0,Finn,6576,8257,1.255627
1,Jake,4757,7210,1.515661
2,Princess Bubblegum,1135,3364,2.963877
3,Ice King,919,3815,4.151251
4,Marceline,481,1647,3.424116


In [ ]:
# keeps only a set of characters
character_list = ['Finn', 'Jake', 'Ice King', 'Princess Bubblegum', 'Marceline', 'BMO']